In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import DeltaTable
from pyspark.sql.window import Window

In [0]:
customer_df = spark.read.table('main.silver_al.customers')
order_df = spark.read.table('main.silver_al.orders')
product_df = spark.read.table('main.silver_al.products')
category_df = spark.read.table('main.silver_al.categories')
payment_df = spark.read.table('main.silver_al.payments')

### Customer Monthly Revenue with Top Customer Ranking

In [0]:
report_1 = customer_df.alias('c').join(order_df.alias('o'),'customer_id').join(payment_df.alias('p'),'order_id')\
    .withColumn('month',date_format(col('order_date'),'yyyy-MM'))\
    .groupBy('c.customer_name','month').agg(sum('p.amount').alias('total_revenue')\
    ,countDistinct('o.order_id').alias('total_orders'))

w = Window.partitionBy('month').orderBy(col('total_revenue').desc())
report_1 = report_1.withColumn('rank_by_customer',dense_rank().over(w))

### Top Selling Products with Ranking

In [0]:
report_2 = product_df.alias('p').join(order_df.alias('o'),'product_id').groupBy('p.product_id','p.product_name')\
    .agg(sum('o.amount').alias('total_revenue'),
     sum(col('o.amount')/col('p.price')).alias('total_quantity_sold'))

w = Window.orderBy(col('total_revenue').desc())
report_2 = report_2.withColumn('rank_by_product',dense_rank().over(w))

### category performance report

In [0]:
report_3 = order_df.alias('o').join(product_df.alias('p'),'product_id').join(category_df.alias('c'),'category_id')\
    .groupBy('c.category_id','c.category_name')\
    .agg(sum('o.amount').alias('total_revenue'),countDistinct('o.order_id').alias('total_orders')
    ,countDistinct('o.customer_id').alias('total_customers'))

w = Window.orderBy(col('total_revenue').desc())
report_3 = report_3.withColumn('rank_by_revenue',dense_rank().over(w))

In [0]:
report_1.write.format('delta').mode('overwrite').saveAsTable('main.gold_al.report_1')
report_2.write.format('delta').mode('overwrite').saveAsTable('main.gold_al.report_2')
report_3.write.format('delta').mode('overwrite').saveAsTable('main.gold_al.report_3')